In [1]:
#IMPORTS#
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from library.sb_utils import save_file

In [2]:
#LOADING DATA#
sales_data = pd.read_csv('C:/Users/Anthony/GitHub/Capstone-Two/Data/raw_data.csv')

In [3]:
#ENSURING THERE ARE NO EMPTY CELLS#
missing = pd.concat([sales_data.isnull().sum(), 100 * sales_data.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count', ascending = False)

,count,%
month,0,0.0
customer_type,0,0.0
order_id,0,0.0
customer_id,0,0.0
customer_name,0,0.0
product_id,0,0.0
product_price,0,0.0
ordered_item_quantity,0,0.0
total_sales,0,0.0


In [4]:
sales_data.columns

Index(['month', 'customer_type', 'order_id', 'customer_id', 'customer_name',
       'product_id', 'product_price', 'ordered_item_quantity', 'total_sales'],
      dtype='object')

In [5]:
#DROP CUSTOMER NAME#
sales_data = sales_data.drop('customer_name', axis=1)

In [6]:
sales_data.shape

(15691, 8)

In [7]:
#CHANGING PRODUCT ID TO PRODUCT NAME
sales_data['product_id'].unique()

array([   9503187985, 5780581712024, 5489893605528, 3554956247133,
       3544999428189,             0, 6178006302872, 6984424063128],
      dtype=int64)

In [8]:
sales_data['product_id'] = sales_data['product_id'].replace([6984424063128, 5780581712024, 3544999428189, 5489893605528, 9503187985, 3554956247133], 
                                 ['Starter Bundle with Coaching', 'Starter Bundle', "Lugol's 5%", 'HaloDetox', 'Thiodine', 'ThyroConvert'])

In [9]:
sales_data.rename(columns = {'product_id': 'product_name'}, inplace = True)
sales_data.columns

Index(['month', 'customer_type', 'order_id', 'customer_id', 'product_name',
       'product_price', 'ordered_item_quantity', 'total_sales'],
      dtype='object')

In [10]:
sales_data.shape

(15691, 8)

In [11]:
sales_data = sales_data[sales_data['product_name'].isin(
    ['Starter Bundle with Coaching', 'Starter Bundle', "Lugol's 5%", 'HaloDetox', 'Thiodine', 'ThyroConvert'])]
sales_data['product_name'].unique()

array(['Thiodine', 'Starter Bundle', 'HaloDetox', 'ThyroConvert',
       "Lugol's 5%", 'Starter Bundle with Coaching'], dtype=object)

In [12]:
sales_data.shape

(14956, 8)

In [13]:
#TURNING MONTH COLUMN DTYPE INTO DATETIME#
sales_data['month'].dtype

dtype('O')

In [14]:
sales_data['month'] =  pd.to_datetime(sales_data['month'], format='%Y-%m')

In [15]:
sales_data['month'].dtype

dtype('<M8[ns]')

In [16]:
#MAKING DATAFRAME OF ONLY CUSTOMERS FIRST ORDERS#
sales_data['customer_type'].unique()

array(['Returning', 'First-time'], dtype=object)

In [17]:
first_orders = sales_data[sales_data['customer_type'] == 'First-time']

In [18]:
first_orders['customer_type'].unique()

array(['First-time'], dtype=object)

In [19]:
first_orders.shape

(12399, 8)

In [20]:
#MAKING DATAFRAME OF ONLY RETURNING CUSTOMER'S ORDERS#
returning_orders = sales_data[sales_data['customer_type'] == 'Returning']

In [21]:
returning_orders['customer_type'].unique()

array(['Returning'], dtype=object)

In [22]:
returning_orders.shape

(2557, 8)

In [23]:
#THERE SHOULD BE THE SAME NUMBER OF FIRST ORDER CUSTOMER IDs AND ORDER IDs#
len(first_orders['order_id'].unique())

11712

In [24]:
#CONFIRMED#
len(first_orders['customer_id'].unique())

11712

In [25]:
#MAKING DATAFRAME OF ONLY PEOPLE WHO ENDING UP AS RETURNING CUSTOMERS#
first_orders_returned = first_orders[first_orders['customer_id'].isin(returning_orders['customer_id'])]
first_orders_returned.shape

(1260, 8)

In [26]:
#INVERSE OF PREVIOUS STEP TO MAKE SURE EACH IN RETURNING ORDERS DATEFRAME LINES UP WITH A FIRST ORDER#
returning_orders = returning_orders[returning_orders['customer_id'].isin(first_orders_returned['customer_id'])]
len(returning_orders['customer_id'].unique())

1118

In [27]:
returning_orders.shape

(1915, 8)

In [28]:
#MAKING A DATAFRAME OF EVERY ORDER A PERSON MADE WHO MADE MORE THAN ONE ORDER IN THIS TIME#
returning_orders_merged = pd.concat([first_orders_returned, returning_orders])
returning_orders_merged.shape

(3175, 8)

In [29]:
#MAKING A DATAFRAME FOR CUSTOMERS WHO ONLY MADE THEIR FIRST ORDER IN THIS TIME FRAME#
one_time_customers = first_orders[~first_orders['customer_id'].isin(returning_orders['customer_id'])]
one_time_customers.shape

(11139, 8)

In [30]:
datapath = 'C:/Users/Anthony/GitHub/Capstone-Two/Data'
save_file(one_time_customers, 'one_time_customers.csv', datapath)

Writing file.  "C:/Users/Anthony/GitHub/Capstone-Two/Data\one_time_customers.csv"


In [31]:
save_file(first_orders_returned, 'first_orders_returned.csv', datapath)

Writing file.  "C:/Users/Anthony/GitHub/Capstone-Two/Data\first_orders_returned.csv"


In [32]:
save_file(returning_orders, 'returning_orders.csv', datapath)

Writing file.  "C:/Users/Anthony/GitHub/Capstone-Two/Data\returning_orders.csv"


In [33]:
save_file(returning_orders_merged, 'returning_orders_merged.csv', datapath)

Writing file.  "C:/Users/Anthony/GitHub/Capstone-Two/Data\returning_orders_merged.csv"


There are now 4 DataFrames:

--one_time_customers: customers who bought one item from the store and never returned

--first_orders_returned: customers who bought one item and ended up buying something again

--returning_orders: customers who had already made an order on the store previously

--returning_orders_merged: all of the orders from customers who made more than one purchase

Going into this stage, I already knew the data was going to be pretty clean since I got the data from Google Analytics. My goal was to organize the data and make sure I had what I wanted in 4 easy-to-access DataFrames. Initially, I was presented with over 15000 rows of data with 9 columns. Once I knew that we didn't have any empty cells, I decided that the customer name column was not useful so I dropped that immediately. Next, I changed the product id column to product name and turned all of the numbers into an item name. I also dropped any rows that had a product name not in the list I specified, so any mistakes there were removed. After that, I used the pandas 'to_datetime' method to change the month column from an object to a datetime object to I could then sort the data by data in any future step. Initially at this point I was going to drop any duplicate order id's, but then I realized that the DataFrame has to separate the data into 2 rows if a person ordered two different products in their order. Finally, I created the four datafreames specified above to prepare myself for the next step. I did this because this isn't the beginning or the end of our data, so I was to make sure that I'm not counting returning customers when I can't even tell what their first order was.